# Классификация текстовых комментариев на позитивные и негативные


## Данные

В наличии один датасет со столбцами *text*, в котором содержатся текст комментария и  *toxic* — целевой признак (токсичный или не токсичный даный комментарий).

## Задача
Необходимо обучить модель классифицировать комментарии на позитивные и негативные.

## Используемые библиотеки
*pandas, sklearn, catboost, nltk, transformers, torch, tqdm, hyperopt*


In [1]:
#Заблокируем предупреждения
import warnings
warnings.filterwarnings('ignore')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  


In [2]:
# Импортируем необходимые библиотеки
import pandas as pd
import sklearn
import datetime
import time
from tqdm import notebook
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt 
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
import torch
import nltk
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

# 1. Подготовка

## 1.1. Загрузка и изучение данных

In [3]:
corpus = pd.read_csv("toxic_comments.csv")

Посмотрим на основную информацию из датасета

In [4]:
print(corpus.shape)
print(corpus.info())
corpus.head()

(159571, 2)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB
None


,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


Проверим дубликаты

In [5]:
corpus.duplicated().sum()

0

Проверим дисбаланс классов

In [6]:
corpus["toxic"].value_counts()

0    143346
1     16225
Name: toxic, dtype: int64

Итак мы видим, датасет состоящий из 159571 строки и 2 столбцов. Один столбец представлен в виде строк (object). Второй столбец представлен целыми числами - 0 и 1. Можно выделить следующие особенности даноого датасета:
* Пропусков нет
* Дубликатов нет
* Имеется дисбаланс классов - Позитивных - 10.2%, негативных - 89.8%

## 1.2. Лемматизация и очистка текста

Разделим корпус на обучающую, валидационную и тестовую выборки

In [7]:
data_train, data_test = train_test_split(corpus, test_size=0.2, random_state=12345)
data_train = data_train.reset_index(drop=True)
data_test = data_test.reset_index(drop=True)

Поскольку для поиска оптимальных параметров моделей я использую подхjды на основе кросс-валидации, валидационая выборка не нужна

In [8]:
texts_train = data_train["text"]
texts_test = data_test["text"]
texts_train = texts_train.reset_index(drop=True)

Импортируем дополнительные библиотеки

In [9]:
import numpy as np
import re
import nltk
nltk.download('wordnet')
from sklearn.datasets import load_files
nltk.download('stopwords')
import pickle
from nltk.corpus import stopwords

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Борис\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Борис\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Напишем функцию для очистки и лемматизации текста

In [10]:
from nltk.stem import WordNetLemmatizer
def texts_prepare(corpus):
    texts_lemm = []
    stemmer = WordNetLemmatizer()
    for sen in notebook.tqdm(range(0, len(corpus))):
      # Удалим все специальные символы
        document = re.sub(r'\W', ' ', str(corpus[sen]))
    
    # Удалим все одиночные символы
        document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
    
    # Удалим одиночные символы с самого начала
        document = re.sub(r'\^[a-zA-Z]\s+', ' ', document) 
    
    # Заменим несколько пробелов одним пробелом
        document = re.sub(r'\s+', ' ', document, flags=re.I)
    
    # Удалим префикс "b"
        document = re.sub(r'^b\s+', '', document)
    
    # Переведем все буквы в нижний регистр
        document = document.lower()
    
    # Проведем лемматизацию
        document = document.split()

        document = [stemmer.lemmatize(word) for word in document]
        document = ' '.join(document)
    
        texts_lemm.append(document)
    return texts_lemm

Проведем очистку и лемматизацию текста

In [11]:
texts_train_lemm = texts_prepare(texts_train)
texts_test_lemm = texts_prepare(texts_test)

In [13]:
# texts_train_lemm.to_csv('texts_train_lemm', index = False)
# texts_test_lemm.to_csv('texts_test_lemm', index = False)

In [14]:
# texts_train_lemm = pd.read_csv('tweets_lemm_train.csv')
# texts_test_lemm = pd.read_csv('texts_test_lemm.csv')

Проведем тестирование для определение лучших параметров векторизации

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
stopwords = set(nltk_stopwords.words('english'))

In [13]:
model_param = pd.DataFrame([])
for max_features in notebook.tqdm(range(8000, 12000, 1000)):
    for min_df in notebook.tqdm(range(5, 8, 1)):
        for max_df in notebook.tqdm(np.arange(0.6, 0.9, 0.1)):
            count_tf_idf = TfidfVectorizer(max_features=max_features, min_df=min_df, max_df=max_df, stop_words=stopwords)
            tf_idf = count_tf_idf.fit_transform(texts_train_lemm)
            tf_idf_test = count_tf_idf.transform(texts_test_lemm)
            
            target_train = data_train['toxic']
            target_test = data_test['toxic']
            features_train = tf_idf
            features_test = tf_idf_test
            
            LR = LogisticRegression(random_state=12345, penalty = "l1")
            LR.fit(features_train, target_train)
            y_predict = LR.predict(features_test)
                   
            model_param = model_param.append(pd.DataFrame({"max_features": [max_features],
                                                           "min_df": [min_df],
                                                           "max_df": [max_df],
                                                           "f1_score": [f1_score(target_test, y_predict)] 
                                               }))

Определим лучшие параметры для векоризации

In [14]:
model_param = model_param.sort_values("f1_score", ascending = False)
model_param.head()

,max_features,min_df,max_df,f1_score
0,10000,5,0.6,0.776192
0,10000,5,0.7,0.776192
0,10000,5,0.9,0.776192
0,10000,5,0.8,0.776192
0,10000,7,0.8,0.775922


Проведем векторизацию текстов с данными параметрами

In [15]:
count_tf_idf = TfidfVectorizer(max_features=10000, min_df=5, max_df=0.7, stop_words=stopwords)

In [16]:
tf_idf = count_tf_idf.fit_transform(texts_train_lemm)
tf_idf_test = count_tf_idf.transform(texts_test_lemm)

In [17]:
print("Размер обучающей матрицы:", tf_idf.shape)
print("Размер тестовой матрицы:", tf_idf_test.shape)

Размер обучающей матрицы: (127656, 10000)
Размер тестовой матрицы: (31915, 10000)


# 2. Обучение

Подготовим выборки для обучения

In [18]:
target_train = data_train['toxic']
target_test = data_test['toxic']

features_train = tf_idf
features_test = tf_idf_test

# 2.1. Логистическая регрессия

Обучим модель логистической регрессии на дефолтных параметрах и сделаем предсказания

In [19]:
LR = LogisticRegression(random_state=12345, penalty = "l1", max_iter = 100)
LR.fit(features_train, target_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l1',
                   random_state=12345, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

Посмотрим на f1 тестовой выборки

In [20]:
y_predict_test = LR.predict(features_test)
print("f1_score тест", f1_score(target_test, y_predict_test))

f1_score тест 0.776192133658197


Мы добились необходимого результата, но попробуем его улучшить с помощью подбора параметров средствами библиотеки hyperopt

Код выполняется долго, поэтому я его закомментировал, а результат перенес в словарь best и в модель

In [39]:
from hyperopt import tpe
from hyperopt import STATUS_OK
from hyperopt import Trials
from hyperopt import hp
from hyperopt import fmin

N_FOLDS = 10
MAX_EVALS = 50



def objective(params, n_folds = N_FOLDS):
    """Objective function for Logistic Regression Hyperparameter Tuning"""

    # Perform n_fold cross validation with hyperparameters
    # Use early stopping and evaluate based on ROC AUC

    clf = LogisticRegression(**params,random_state=0,verbose =0)
    scores = cross_val_score(clf, features_train, target_train, cv=5, scoring='f1_macro')

    # Extract the best score
    best_score = max(scores)

    # Loss must be minimized
    loss = 1 - best_score

    # Dictionary with information for evaluation
    return {'loss': loss, 'params': params, 'status': STATUS_OK}

space = {
    'class_weight': hp.choice('class_weight', [None, "balanced"]),
    'penalty': hp.choice('penalty', ["l1", "l2"]),
    'warm_start' : hp.choice('warm_start', [True, False]),
    'tol' : hp.uniform('tol', 0.00001, 0.0001),
    'C' : hp.uniform('C', 0.05, 3),
    'max_iter' : hp.choice('max_iter', range(200,1000))
}


bayes_trials = Trials()

# Optimize
best = fmin(fn = objective, space = space, algo = tpe.suggest, max_evals = MAX_EVALS, trials = bayes_trials)


100%|███████| 50/50 [19:41<00:00, 23.62s/trial, best loss: 0.12033248855872791]


Посмотрим на лучшие параметры

In [43]:
best

{'C': 2.0587393315520224,
 'class_weight': 0,
 'max_iter': 248,
 'penalty': 0,
 'tol': 5.593994521218677e-05,
 'warm_start': 1}

Обучим модель с лучшими параметрами и получим предстказания

In [44]:
LR = LogisticRegression(random_state=12345, C = 2.0587393315520224, penalty = "l1", max_iter = 248, tol = 5.593994521218677e-05, warm_start = False, class_weight = None)
LR.fit(features_train, target_train)

y_predict_test = LR.predict(features_test)

print("f1_score тест", f1_score(target_test, y_predict_test))

f1_score тест 0.7797131147540984


Модель соответсвует необходимым требованиям, но попробуем ее еще улучшить через изменение порога классификации

In [45]:
model = LogisticRegression(random_state=12345, C = 2.0587393315520224, penalty = "l1", max_iter = 248, tol = 5.593994521218677e-05, warm_start = False, class_weight = None)
model.fit(features_train, target_train)
probabilities_valid = model.predict_proba(features_test)
probabilities_one_valid = probabilities_valid[:, 1]

for threshold in np.arange(0.35, 0.45, 0.01):
    predicted_test = probabilities_one_valid > threshold
    precision = precision_score(target_test, predicted_test)
    recall = recall_score(target_test, predicted_test)
    f1 = f1_score(target_test, predicted_test)
    print("Порог = {:.2f} | Точность = {:.2f}, Полнота = {:.2f}, f1 = {:.4f}".format(threshold, precision, recall, f1))

Порог = 0.35 | Точность = 0.82, Полнота = 0.75, f1 = 0.7818
Порог = 0.36 | Точность = 0.82, Полнота = 0.75, f1 = 0.7819
Порог = 0.37 | Точность = 0.83, Полнота = 0.74, f1 = 0.7829
Порог = 0.38 | Точность = 0.83, Полнота = 0.74, f1 = 0.7841
Порог = 0.39 | Точность = 0.83, Полнота = 0.74, f1 = 0.7838
Порог = 0.40 | Точность = 0.84, Полнота = 0.74, f1 = 0.7842
Порог = 0.41 | Точность = 0.84, Полнота = 0.73, f1 = 0.7838
Порог = 0.42 | Точность = 0.84, Полнота = 0.73, f1 = 0.7824
Порог = 0.43 | Точность = 0.85, Полнота = 0.73, f1 = 0.7840
Порог = 0.44 | Точность = 0.85, Полнота = 0.73, f1 = 0.7840
Порог = 0.45 | Точность = 0.86, Полнота = 0.72, f1 = 0.7833


Лучший f1 = 0.784, достигается при пороге = 0.4

# 2.2. Стохастический градиентный спуск

Посмотрим как покажет сябя стахостический градиентный спуск

In [46]:
from sklearn.linear_model import SGDClassifier
SGDC = SGDClassifier(loss = "log", penalty = "l1", random_state=12345, alpha=0.00001)
SGDC.fit(features_train, target_train)

SGDClassifier(alpha=1e-05, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='log', max_iter=1000,
              n_iter_no_change=5, n_jobs=None, penalty='l1', power_t=0.5,
              random_state=12345, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

In [47]:
y_predict_test = SGDC.predict(features_test)

print("f1_score тест", f1_score(target_test, y_predict_test))

f1_score тест 0.7706582633053222


Подберем лучшие параметры

Код выполняется долго, поэтому я его закомментировал, а результат перенес в словарь best и в модель

In [52]:
N_FOLDS = 10
MAX_EVALS = 50



def objective(params, n_folds = N_FOLDS):
    """Objective function for Logistic Regression Hyperparameter Tuning"""

    # Perform n_fold cross validation with hyperparameters
    # Use early stopping and evaluate based on ROC AUC

    clf = SGDClassifier(**params,random_state=12345, loss = "log", verbose = 0)
    scores = cross_val_score(clf, features_train, target_train, cv=5, scoring='f1_macro')

    # Extract the best score
    best_score = max(scores)

    # Loss must be minimized
    loss = 1 - best_score

    # Dictionary with information for evaluation
    return {'loss': loss, 'params': params, 'status': STATUS_OK}

space = {
    'class_weight': hp.choice('class_weight', [None, "balanced"]),
    'penalty': hp.choice('penalty', ["l1", "l2"]),
    'warm_start' : hp.choice('warm_start', [True, False]),
    'fit_intercept' : hp.choice('fit_intercept', [True, False]),
    'alpha' : hp.uniform('alpha', 0.0000001, 0.0001),
    'tol' : hp.uniform('tol', 0.00001, 0.0001),
    'max_iter' : hp.choice('max_iter', range(5, 1000))
}


bayes_trials = Trials()

# Optimize
best = fmin(fn = objective, space = space, algo = tpe.suggest, max_evals = MAX_EVALS, trials = bayes_trials)


100%|███████| 50/50 [15:52<00:00, 19.06s/trial, best loss: 0.12617593607183897]


Посмотрим на лучшие параметры

In [53]:
best

{'alpha': 1.2573330481208354e-05,
 'class_weight': 0,
 'fit_intercept': 1,
 'max_iter': 679,
 'penalty': 0,
 'tol': 9.088054857563279e-05,
 'warm_start': 1}

Обучим модель с лучшими параметрами и получим предстказания

In [64]:
SGDC = SGDClassifier(fit_intercept = True, alpha = 3.328059888505736e-06, loss = "log", random_state=12345, penalty = "l1", max_iter = 378, tol = 1.3032360763407112e-05, warm_start = True, class_weight = None)
SGDC.fit(features_train, target_train)

y_predict_test = SGDC.predict(features_test)

print("f1_score тест", f1_score(target_test, y_predict_test))

f1_score тест 0.7770961145194274


Модель соответсвует необходимым требованиям, но попробуем ее еще улучшить через изменение порога классификации

In [66]:
model = SGDC
model.fit(features_train, target_train)
probabilities_valid = model.predict_proba(features_test)
probabilities_one_valid = probabilities_valid[:, 1]

for threshold in np.arange(0.3, 0.4, 0.01):
    predicted_test = probabilities_one_valid > threshold
    precision = precision_score(target_test, predicted_test)
    recall = recall_score(target_test, predicted_test)
    f1 = f1_score(target_test, predicted_test)
    print("Порог = {:.2f} | Точность = {:.2f}, Полнота = {:.2f}, f1 = {:.4f}".format(threshold, precision, recall, f1))

Порог = 0.30 | Точность = 0.78, Полнота = 0.78, f1 = 0.7791
Порог = 0.31 | Точность = 0.79, Полнота = 0.77, f1 = 0.7801
Порог = 0.32 | Точность = 0.79, Полнота = 0.77, f1 = 0.7816
Порог = 0.33 | Точность = 0.80, Полнота = 0.76, f1 = 0.7815
Порог = 0.34 | Точность = 0.80, Полнота = 0.76, f1 = 0.7810
Порог = 0.35 | Точность = 0.81, Полнота = 0.76, f1 = 0.7814
Порог = 0.36 | Точность = 0.81, Полнота = 0.75, f1 = 0.7818
Порог = 0.37 | Точность = 0.82, Полнота = 0.75, f1 = 0.7831
Порог = 0.38 | Точность = 0.82, Полнота = 0.75, f1 = 0.7820
Порог = 0.39 | Точность = 0.83, Полнота = 0.74, f1 = 0.7827
Порог = 0.40 | Точность = 0.83, Полнота = 0.74, f1 = 0.7826


Лучший f1 = 0.783, достигается при пороге = 0.37

# 2.3. BERT

Посмотрим на что способна нейронная сеть

Импортируем библиотеки

In [68]:
import torch
import transformers as ppb
import warnings
warnings.filterwarnings('ignore')

Загрузим корпус и для ускорения работы оставим 3000 строк

In [69]:
# df = pd.read_csv("toxic_comments.csv")
# batch_1 = df.sample(3000).reset_index(drop=True)

Поскольку я выбираю 3000 СЛУЧАЙНЫХ текстов из корпуса, то результат получается не вполне стабильный. А увеличить выборку я не могу из-за ограниченности русурсов. Поэтому я сохраню выборку в csv из которого она будет подтягиваться при работе проекта. А формирование выборки закомментирую.

In [70]:
# batch_1.to_csv('batch_1.csv',  index = False)

In [71]:
batch_1 = pd.read_csv("batch_1.csv")

In [72]:
batch_1

,text,toxic
0,Evidently it is one rule for unfavorable accou...,1
1,Where am I? What happened?,0
2,E. E. Cummings \n\nE. E. Cummings' name should...,0
3,"""\n\nYou like the article? Remember, Jehovah i...",0
4,"Anonymous is a hacktivist group, they have hac...",0
...,...,...
2995,"""\n\n Lololol \n\nEvil. Haha. Well I just saw ...",0
2996,Edit wars and questions \n\nHi - you posted on...,0
2997,Thanks for including in the disscusion. Neutra...,0
2998,Is Denise Lor still alive and if so where does...,0


Посмотрим на баланс классов

In [73]:
batch_1["toxic"].value_counts()

0    2681
1     319
Name: toxic, dtype: int64

Те же 10%

Загрузим предварительно обученную модель

In [74]:
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

#Для получения полноценной BERT можно раскоментировать следующую строку:
#model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

#Предварительно обученная модель токенизатор
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

Проведем токенизацию каждого выражения в нашей выборке

In [75]:
tokenized = batch_1["text"].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

Token indices sequence length is longer than the specified maximum sequence length for this model (713 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (903 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (969 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1597 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (694 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for th

Token indices sequence length is longer than the specified maximum sequence length for this model (845 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (705 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1080 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (707 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (868 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for th

Посмотрм на токены

In [76]:
tokenized

0       [101, 15329, 2009, 2003, 2028, 3627, 2005, 489...
1       [101, 2073, 2572, 1045, 1029, 2054, 3047, 1029...
2       [101, 1041, 1012, 1041, 1012, 20750, 1041, 101...
3       [101, 1000, 2017, 2066, 1996, 3720, 1029, 3342...
4       [101, 10812, 2003, 1037, 20578, 29068, 2923, 2...
                              ...                        
2995    [101, 1000, 8840, 4135, 4135, 2140, 4763, 1012...
2996    [101, 10086, 5233, 1998, 3980, 7632, 1011, 201...
2997    [101, 4283, 2005, 2164, 1999, 1996, 4487, 4757...
2998    [101, 2003, 15339, 8840, 2099, 2145, 4142, 199...
2999    [101, 1000, 2115, 4431, 2000, 1037, 9949, 5549...
Name: text, Length: 3000, dtype: object

После токенизации tokenized представляет собой список предложений - каждое предложение представлено в виде списка токенов. Мы хотим, чтобы Берт обрабатывал наши примеры все сразу (как один пакет). Просто так будет быстрее. По этой причине нам нужно заполнить все списки до одного размера, чтобы мы могли представить входные данные как один 2-d массив, а не список списков (разной длины).

In [77]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

In [78]:
print("Массив имеет размер", np.array(padded).shape)

Массив имеет размер (3000, 1599)


Из опыта мы знаем, что обрабатывать вектора длинной более 50 сложно и ядро умирает. Поэтому ограничим этот размер

In [79]:
padded = padded[:, :50]

In [80]:
print("Новый размер массива", np.array(padded).shape)

Новый размер массива (3000, 50)


Теперь поясним модели, что нули не несут значимой информации. Это нужно для компоненты модели, которая называется «внимание» (англ. attention). Отбросим эти токены и «создадим маску» для действительно важных токенов, то есть укажем нулевые и не нулевые значения

In [81]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(3000, 50)

Начнём преобразование текстов в эмбеддинги. Это может занять несколько минут, поэтому подключим библиотеку tqdm (араб. taqadum, تقدّم, «прогресс»). Она нужна, чтобы наглядно показать индикатор прогресса. В Jupyter применим функцию notebook() из этой библиотеки:

Эмбеддинги модель BERT создаёт батчами. Чтобы хватило оперативной памяти, сделаем размер батча небольшим. Мы срежем только ту часть выходных данных, которая нам нужна. Это выход, соответствующий первому знаку каждого предложения. Способ, которым Берт делает классификацию предложений, заключается в том, что он добавляет маркер под названием [CLS] (для классификации) в начале каждого предложения. Вывод, соответствующий этому маркеру, можно рассматривать как вложение для всего предложения.

In [82]:
from tqdm import notebook
batch_size = 100
embeddings = []
for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
        batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)]) 
        attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])
        
        with torch.no_grad():
            batch_embeddings = model(batch, attention_mask=attention_mask_batch)
        
        embeddings.append(batch_embeddings[0][:,0,:].numpy())

Соберём все эмбеддинги в матрицу признаков вызовом функции concatenate()

In [83]:
features = np.concatenate(embeddings)

Выделим целевые признаки

In [84]:
target = batch_1["toxic"]

Сформируем обучающую и тестовую выбоки

In [85]:
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.2, random_state=12345)

Посмотрим на результаты работы логистической регрессии на дефолтных параметрах

In [86]:
lr_clf = LogisticRegression()
lr_clf.fit(features_train, target_train)
y_predict = lr_clf.predict(features_test)
from sklearn.metrics import classification_report, confusion_matrix, f1_score
print(f1_score(target_test, y_predict))

0.574468085106383


Посмотрим на f1 при изменении порога классификации

In [87]:
model = lr_clf
model.fit(features_train, target_train)
probabilities_valid = model.predict_proba(features_test)
probabilities_one_valid = probabilities_valid[:, 1]

for threshold in np.arange(0.35, 0.45, 0.01):
    predicted_test = probabilities_one_valid > threshold
    precision = precision_score(target_test, predicted_test)
    recall = recall_score(target_test, predicted_test)
    f1 = f1_score(target_test, predicted_test)
    print("Порог = {:.2f} | Точность = {:.2f}, Полнота = {:.2f}, f1 = {:.4f}".format(threshold, precision, recall, f1))

Порог = 0.35 | Точность = 0.55, Полнота = 0.59, f1 = 0.5714
Порог = 0.36 | Точность = 0.54, Полнота = 0.57, f1 = 0.5586
Порог = 0.37 | Точность = 0.54, Полнота = 0.57, f1 = 0.5586
Порог = 0.38 | Точность = 0.58, Полнота = 0.57, f1 = 0.5794
Порог = 0.39 | Точность = 0.57, Полнота = 0.54, f1 = 0.5524
Порог = 0.40 | Точность = 0.60, Полнота = 0.52, f1 = 0.5545
Порог = 0.41 | Точность = 0.60, Полнота = 0.52, f1 = 0.5545
Порог = 0.42 | Точность = 0.60, Полнота = 0.52, f1 = 0.5545
Порог = 0.43 | Точность = 0.61, Полнота = 0.52, f1 = 0.5600
Порог = 0.44 | Точность = 0.61, Полнота = 0.52, f1 = 0.5600
Порог = 0.45 | Точность = 0.63, Полнота = 0.50, f1 = 0.5567


Лучший f1 достигается при пороге = 0.38

Попробуем улучшить результат работы модели с помощью подбора параметров средствами библиотеки hyperopt

Код выполняется долго, поэтому я его закомментировал, а результат перенес в словарь best и в модель

In [88]:
from hyperopt import tpe
from hyperopt import STATUS_OK
from hyperopt import Trials
from hyperopt import hp
from hyperopt import fmin

N_FOLDS = 10
MAX_EVALS = 50



def objective(params, n_folds = N_FOLDS):
    """Objective function for Logistic Regression Hyperparameter Tuning"""

    # Perform n_fold cross validation with hyperparameters
    # Use early stopping and evaluate based on ROC AUC

    clf = LogisticRegression(**params, random_state=12345, verbose =0)
    scores = cross_val_score(clf, features_train, target_train, cv=3, scoring='f1_macro')

    # Extract the best score
    best_score = max(scores)

    # Loss must be minimized
    loss = 1 - best_score

    # Dictionary with information for evaluation
    return {'loss': loss, 'params': params, 'status': STATUS_OK}

space = {
    'class_weight': hp.choice('class_weight', [None, "balanced"]),
    'penalty': hp.choice('penalty', ["l1", "l2"]),
    'warm_start' : hp.choice('warm_start', [True, False]),
    'tol' : hp.uniform('tol', 0.00001, 0.0001),
    'C' : hp.uniform('C', 0.05, 3),
    'max_iter' : hp.choice('max_iter', range(5, 1000))
}


bayes_trials = Trials()

# Optimize
best = fmin(fn = objective, space = space, algo = tpe.suggest, max_evals = MAX_EVALS, trials = bayes_trials)


100%|███████| 50/50 [05:47<00:00,  6.94s/trial, best loss: 0.17363525730180807]


Посмотрим на лучшие параметры

In [89]:
best

{'C': 1.3161314187759485,
 'class_weight': 0,
 'max_iter': 720,
 'penalty': 0,
 'tol': 2.215304976357311e-05,
 'warm_start': 1}

Обучим модель с лучшими параметрамии получим предсказания

In [91]:
LR = LogisticRegression(random_state=12345, C = 1.3161314187759485, penalty = "l1", max_iter = 720, tol = 2.215304976357311e-05, warm_start = True, class_weight = None)
LR.fit(features_train, target_train)

y_predict_test = LR.predict(features_test)

print("f1_score тест", f1_score(target_test, y_predict_test))

f1_score тест 0.553191489361702


Посмотрим на f1 при измеении порога классификации

In [93]:
model = LR
model.fit(features_train, target_train)
probabilities_valid = model.predict_proba(features_test)
probabilities_one_valid = probabilities_valid[:, 1]

for threshold in np.arange(0.35, 0.55, 0.01):
    predicted_test = probabilities_one_valid > threshold
    precision = precision_score(target_test, predicted_test)
    recall = recall_score(target_test, predicted_test)
    f1 = f1_score(target_test, predicted_test)
    print("Порог = {:.2f} | Точность = {:.2f}, Полнота = {:.2f}, f1 = {:.4f}".format(threshold, precision, recall, f1))

Порог = 0.35 | Точность = 0.54, Полнота = 0.57, f1 = 0.5586
Порог = 0.36 | Точность = 0.55, Полнота = 0.56, f1 = 0.5505
Порог = 0.37 | Точность = 0.55, Полнота = 0.56, f1 = 0.5505
Порог = 0.38 | Точность = 0.57, Полнота = 0.56, f1 = 0.5607
Порог = 0.39 | Точность = 0.57, Полнота = 0.56, f1 = 0.5607
Порог = 0.40 | Точность = 0.56, Полнота = 0.54, f1 = 0.5472
Порог = 0.41 | Точность = 0.55, Полнота = 0.52, f1 = 0.5333
Порог = 0.42 | Точность = 0.60, Полнота = 0.52, f1 = 0.5545
Порог = 0.43 | Точность = 0.61, Полнота = 0.52, f1 = 0.5600
Порог = 0.44 | Точность = 0.62, Полнота = 0.52, f1 = 0.5657
Порог = 0.45 | Точность = 0.64, Полнота = 0.52, f1 = 0.5714
Порог = 0.46 | Точность = 0.64, Полнота = 0.52, f1 = 0.5714
Порог = 0.47 | Точность = 0.63, Полнота = 0.50, f1 = 0.5567
Порог = 0.48 | Точность = 0.62, Полнота = 0.48, f1 = 0.5417
Порог = 0.49 | Точность = 0.63, Полнота = 0.48, f1 = 0.5474
Порог = 0.50 | Точность = 0.65, Полнота = 0.48, f1 = 0.5532
Порог = 0.51 | Точность = 0.65, Полнота 

Лучший f1 достигается при пороге = 0.45

Посмотрим как покажет сябя стахостический градиентный спуск

In [94]:
from sklearn.linear_model import SGDClassifier
SGDC = SGDClassifier(loss = "log", penalty = "l1", random_state=12345, alpha=0.0001)
SGDC.fit(features_train, target_train)

SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='log', max_iter=1000,
              n_iter_no_change=5, n_jobs=None, penalty='l1', power_t=0.5,
              random_state=12345, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

In [95]:
y_predict_test = SGDC.predict(features_test)

print("f1_score тест", f1_score(target_test, y_predict_test))

f1_score тест 0.5607476635514018


In [102]:
model = SGDC
model.fit(features_train, target_train)
probabilities_valid = model.predict_proba(features_test)
probabilities_one_valid = probabilities_valid[:, 1]

for threshold in np.arange(0.65, 0.85, 0.01):
    predicted_test = probabilities_one_valid > threshold
    precision = precision_score(target_test, predicted_test)
    recall = recall_score(target_test, predicted_test)
    f1 = f1_score(target_test, predicted_test)
    print("Порог = {:.2f} | Точность = {:.2f}, Полнота = {:.2f}, f1 = {:.4f}".format(threshold, precision, recall, f1))

Порог = 0.65 | Точность = 0.19, Полнота = 0.83, f1 = 0.3030
Порог = 0.66 | Точность = 0.19, Полнота = 0.83, f1 = 0.3030
Порог = 0.67 | Точность = 0.19, Полнота = 0.83, f1 = 0.3030
Порог = 0.68 | Точность = 0.19, Полнота = 0.83, f1 = 0.3030
Порог = 0.69 | Точность = 0.19, Полнота = 0.83, f1 = 0.3041
Порог = 0.70 | Точность = 0.19, Полнота = 0.83, f1 = 0.3041
Порог = 0.71 | Точность = 0.19, Полнота = 0.83, f1 = 0.3041
Порог = 0.72 | Точность = 0.19, Полнота = 0.83, f1 = 0.3041
Порог = 0.73 | Точность = 0.19, Полнота = 0.83, f1 = 0.3041
Порог = 0.74 | Точность = 0.19, Полнота = 0.83, f1 = 0.3041
Порог = 0.75 | Точность = 0.19, Полнота = 0.83, f1 = 0.3041
Порог = 0.76 | Точность = 0.18, Полнота = 0.81, f1 = 0.2983
Порог = 0.77 | Точность = 0.18, Полнота = 0.81, f1 = 0.2983
Порог = 0.78 | Точность = 0.18, Полнота = 0.81, f1 = 0.2983
Порог = 0.79 | Точность = 0.18, Полнота = 0.81, f1 = 0.2983
Порог = 0.80 | Точность = 0.18, Полнота = 0.81, f1 = 0.2983
Порог = 0.81 | Точность = 0.18, Полнота 

Лучший f1 при пороге = 0.69

Подбеем лучшие параметры с помощью hyperopt

In [103]:
N_FOLDS = 10
MAX_EVALS = 50



def objective(params, n_folds = N_FOLDS):
    """Objective function for Logistic Regression Hyperparameter Tuning"""

    # Perform n_fold cross validation with hyperparameters
    # Use early stopping and evaluate based on ROC AUC

    clf = SGDClassifier(**params, loss = "log", random_state=12345, verbose = 0)
    scores = cross_val_score(clf, features_train, target_train, cv=3, scoring='f1_macro')

    # Extract the best score
    best_score = max(scores)

    # Loss must be minimized
    loss = 1 - best_score

    # Dictionary with information for evaluation
    return {'loss': loss, 'params': params, 'status': STATUS_OK}

space = {
    'class_weight': hp.choice('class_weight', [None, "balanced"]),
    'penalty': hp.choice('penalty', ["l1", "l2"]),
    'warm_start' : hp.choice('warm_start', [True, False]),
    'fit_intercept' : hp.choice('fit_intercept', [True, False]),
    'alpha' : hp.uniform('alpha', 0.0000001, 0.0001),
    'tol' : hp.uniform('tol', 0.00001, 0.0001),
    'max_iter' : hp.choice('max_iter', range(5, 1000))
}


bayes_trials = Trials()

# Optimize
best = fmin(fn = objective, space = space, algo = tpe.suggest, max_evals = MAX_EVALS, trials = bayes_trials)


100%|███████| 50/50 [02:29<00:00,  2.99s/trial, best loss: 0.17645222212068612]


Посмотрим на лучшие параметры

In [104]:
best

{'alpha': 7.637102721429022e-05,
 'class_weight': 1,
 'fit_intercept': 1,
 'max_iter': 339,
 'penalty': 1,
 'tol': 6.91788699363432e-05,
 'warm_start': 1}

Обучим модель с лучшими параметрами и получим предстказания

In [105]:
SGDC = SGDClassifier(fit_intercept = False, alpha = 3.3211820238627403e-05, loss = "log", random_state=12345, penalty = "l2", max_iter = 339, tol = 6.91788699363432e-05, warm_start = False, class_weight = "balanced")
SGDC.fit(features_train, target_train)

y_predict_test = SGDC.predict(features_test)

print("f1_score тест", f1_score(target_test, y_predict_test))

f1_score тест 0.5


Подберем лучший порог

In [108]:
model = SGDC
model.fit(features_train, target_train)
probabilities_valid = model.predict_proba(features_test)
probabilities_one_valid = probabilities_valid[:, 1]

for threshold in np.arange(0.1, 0.3, 0.01):
    predicted_test = probabilities_one_valid > threshold
    precision = precision_score(target_test, predicted_test)
    recall = recall_score(target_test, predicted_test)
    f1 = f1_score(target_test, predicted_test)
    print("Порог = {:.2f} | Точность = {:.2f}, Полнота = {:.2f}, f1 = {:.4f}".format(threshold, precision, recall, f1))

Порог = 0.10 | Точность = 0.44, Полнота = 0.63, f1 = 0.5152
Порог = 0.11 | Точность = 0.44, Полнота = 0.63, f1 = 0.5152
Порог = 0.12 | Точность = 0.44, Полнота = 0.63, f1 = 0.5152
Порог = 0.13 | Точность = 0.44, Полнота = 0.63, f1 = 0.5152
Порог = 0.14 | Точность = 0.44, Полнота = 0.63, f1 = 0.5152
Порог = 0.15 | Точность = 0.44, Полнота = 0.63, f1 = 0.5152
Порог = 0.16 | Точность = 0.44, Полнота = 0.63, f1 = 0.5152
Порог = 0.17 | Точность = 0.44, Полнота = 0.63, f1 = 0.5152
Порог = 0.18 | Точность = 0.44, Полнота = 0.63, f1 = 0.5152
Порог = 0.19 | Точность = 0.44, Полнота = 0.63, f1 = 0.5152
Порог = 0.20 | Точность = 0.44, Полнота = 0.63, f1 = 0.5152
Порог = 0.21 | Точность = 0.44, Полнота = 0.63, f1 = 0.5152
Порог = 0.22 | Точность = 0.44, Полнота = 0.63, f1 = 0.5152
Порог = 0.23 | Точность = 0.44, Полнота = 0.63, f1 = 0.5152
Порог = 0.24 | Точность = 0.44, Полнота = 0.63, f1 = 0.5152
Порог = 0.25 | Точность = 0.44, Полнота = 0.63, f1 = 0.5152
Порог = 0.26 | Точность = 0.44, Полнота 

Лучший f1 при пороге = 0.1 

# 3. Выводы

1. В процессе работы была проведена очистка и лемматизация данных
2. Далее данные были подготовлены для обучения следующими методами:
   * TF-IDF
   * предобработка моделью BERT (упрощенным вариантом - DistilBert)
3. В проекте были использованы модели Логистическая регрессия и Стохастический градиентный спуск
4. Параметры для моделей подбирались с помощью библиотеки Hyperopt
5. Показатель дополнительно f1 улучшался с помощью изменения порога классификации
5. Наилучшие результаты f1 = 0.7842 были получены на модели логистической регрессии с параметрами определенными с помощью hyperopt и порогом классификации 0.4 